In [2]:
!pip install openai==0.28

In [3]:
import openai
import pandas as pd
from sklearn.metrics import f1_score
import os
import sys
import chardet
import csv

In [4]:
openai.__version__

'0.28.0'

In [5]:
openai.api_key ='key'

In [6]:
import re
def remove_mention(text):
  return re.sub(r'@\w+', '', text)

In [7]:
train = pd.read_csv('train.csv',encoding='ISO-8859-1',engine='python')
test = pd.read_csv('test.csv',encoding='ISO-8859-1',engine='python')

In [8]:
train.shape

(2914, 5)

In [9]:
test.shape

(1956, 5)

In [10]:
train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos


In [11]:
train['Tweet']=train['Tweet'].apply(lambda x: re.sub(r'#SemST','',x).strip())
test['Tweet']=test['Tweet'].apply(lambda x: re.sub(r'#SemST','',x).strip())

In [12]:
train['Tweet'][2]

"@TheView I think our country is ready for a female pres, it can't ever be Hillary"

In [13]:
test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,other
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg


## Zero shot

In [27]:
def classify_stance_zero_shot(tweet, target):
    # Define the prompt for stance detection
    example1 = train[(train['Target']==target) & (train['Stance']=='FAVOR')].sample(1)
    example2 = train[(train['Target']==target) & (train['Stance']=='AGAINST')].sample(1)
    example3 = train[(train['Target']==target) & (train['Stance']=='NONE')].sample(1)
    prompt = f"Detect the stance for the tweet about '{target}', if the author of the given tweet supports '{target}', please label it as 'FAVOR'.\
    If the author of the given tweet is against '{target}', please label it as 'AGAINST'.\
    If the author of the given tweet has a neutral stance or no stance about {target}, please label it as 'NONE'.\
    \nNow detect the stance of the author of this tweet using exactly one word from 'FAVOR','AGAINST','NONE': \
    \ntweet: {tweet}. The author's stance towards {target} is:"

    # Make an API call to GPT-3.5 Turbo for stance detection
    response = openai.Completion.create(
        engine="text-davinci-003", 
        prompt=prompt,
        max_tokens=100,  
        temperature=0.5,  
        stop=None  
    )

    # Extract the generated stance label from the API response
    generated_stance = response['choices'][0]['text'].strip()

    return generated_stance

In [28]:
test_taskA['predicted_stance_zero'] = test_taskA.apply(lambda row: classify_stance_zero_shot(row['Tweet'], row['Target']), axis=1)

In [29]:
f1 = f1_score(test_taskA['Stance'], test_taskA['predicted_stance_zero'], labels=['FAVOR','AGAINST','NONE'],average=None)

# Print the F1 score
print(f"F1 Score: {f1}")

F1 Score: [0.65266559 0.45064378 0.44608879]


# Few shot

In [54]:
def classify_stance(tweet, target):
    # Define the prompt for stance detection
    example1 = train[(train['Target']==target) & (train['Stance']=='FAVOR')].sample(1)
    example2 = train[(train['Target']==target) & (train['Stance']=='AGAINST')].sample(1)
    example3 = train[(train['Target']==target) & (train['Stance']=='NONE')].sample(1)
    prompt = f"Detect the stance for the tweet about '{target}', if the author of the given tweet supports '{target}', please label it as 'FAVOR'.\
    If the author of the given tweet is against '{target}', please label it as 'AGAINST'.\
    If the author of the given tweet has a neutral stance or no stance about {target}, please label it as 'NONE'.\
    Here are some examples: \
    \n1. tweet: {example1['Tweet'].values[0]}. The author's stance towards {target} is: \n{example1['Stance'].values[0]} \
    \n2. tweet: {example2['Tweet'].values[0]}. The author's stance towards {target} is: \n{example2['Stance'].values[0]} \
    \n3. tweet: {example3['Tweet'].values[0]}. The author's stance towards {target} is: \n{example3['Stance'].values[0]} \
    \nNow consider the examples and detect the stance of the author of this tweet, please use excatly one word from 'FAVOR','AGAINST','NONE': \
    \ntweet: {tweet}. The author's stance towards {target} is:"

    # Make an API call to GPT-3.5 Turbo for stance detection
    response = openai.Completion.create(
        engine="text-davinci-003", 
        prompt=prompt,
        max_tokens=100,  
        temperature=0.5,  
        stop=None  
    )

    # Extract the generated stance label from the API response
    generated_stance = response['choices'][0]['text'].strip()

    return generated_stance


In [56]:
test_taskA['predicted_stance'] = test_taskA.apply(lambda row: classify_stance(row['Tweet'], row['Target']), axis=1)

In [57]:
test_taskA

,Tweet,Target,Stance,Opinion Towards,Sentiment,predicted_stance
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos,AGAINST
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,other,NONE
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos,NONE
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,AGAINST
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg,NONE
...,...,...,...,...,...,...
1244,@MetalheadMonty @tom_six I followed him before...,Legalization of Abortion,NONE,2. The tweet does NOT expresses opinion about ...,neg,NONE
1245,"For he who avenges blood remembers, he does no...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other,NONE
1246,Life is sacred on all levels. Abortion does no...,Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,other,AGAINST
1247,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",Legalization of Abortion,AGAINST,1. The tweet explicitly expresses opinion abo...,neg,AGAINST


In [58]:
test_taskA['predicted_stance'].value_counts()

NONE       585
FAVOR      343
AGAINST    321
Name: predicted_stance, dtype: int64

In [59]:
test_taskA.to_csv('gpttaskA.csv')

In [61]:
f1 = f1_score(test_taskA['Stance'], test_taskA['predicted_stance'], labels=['FAVOR','AGAINST','NONE'],average=None)

# Print the F1 score
print(f"F1 Score: {f1}")

F1 Score: [0.72642968 0.59266409 0.53742331]


In [80]:
Hillary=test_taskA[test_taskA['Target']=='Hillary Clinton']
Feminist=test_taskA[test_taskA['Target']=='Feminist Movement']
Abortion=test_taskA[test_taskA['Target']=='Legalization of Abortion']
Atheism=test_taskA[test_taskA['Target']=='Atheism']
Climate=test_taskA[test_taskA['Target']=='Climate Change is a Real Concern']

In [91]:
Climate

,Tweet,Target,Stance,Opinion Towards,Sentiment,predicted_stance
220,Closed door session begins. More after they de...,Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,other,NONE
221,What is the #energiewende agenda? Why are they...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg,NONE
222,@climasphere Stocker: Fish catch potential cou...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg,FAVOR
223,"...a longer memory, and a sterner sense of jus...",Climate Change is a Real Concern,NONE,2. The tweet does NOT expresses opinion about ...,other,NONE
224,"Boni ""Future Sea level rise will not be unifor...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other,FAVOR
...,...,...,...,...,...,...
384,"@LiberalAus - ""All"" you folks #owe me an #offi...",Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,neg,NONE
385,"Maybe we should start ""labelling every #produc...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos,FAVOR
386,If we touch #Antarctica for one more #reason o...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg,NONE
387,Wind power helped stabilize Texas' power grid ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos,FAVOR


In [42]:
def get_f1(df):
    f1 = f1_score(df['Stance'], df['predicted_stance'], labels=['FAVOR','AGAINST','NONE'],average=None).reshape(1,-1)
    f1_df=pd.DataFrame(f1,columns=['FAVOR','AGAINST','NONE'])
    f1_df['AVG']=(f1_df['FAVOR'][0]+f1_df['AGAINST'][0])/2
    print(f1_df)
    return f1_df

In [81]:
get_f1(Hillary)

      FAVOR   AGAINST      NONE       AVG
0  0.815534  0.723247  0.722222  0.769391


,FAVOR,AGAINST,NONE,AVG
0,0.815534,0.723247,0.722222,0.769391


In [82]:
get_f1(Feminist)

      FAVOR   AGAINST      NONE       AVG
0  0.657343  0.554688  0.467836  0.606015


,FAVOR,AGAINST,NONE,AVG
0,0.657343,0.554688,0.467836,0.606015


In [83]:
get_f1(Abortion)

      FAVOR   AGAINST      NONE       AVG
0  0.645161  0.565056  0.491018  0.605109


,FAVOR,AGAINST,NONE,AVG
0,0.645161,0.565056,0.491018,0.605109


In [84]:
get_f1(Climate)

   FAVOR  AGAINST      NONE  AVG
0    0.8      0.6  0.592593  0.7


,FAVOR,AGAINST,NONE,AVG
0,0.8,0.6,0.592593,0.7


In [86]:
get_f1(Atheism)

      FAVOR   AGAINST      NONE       AVG
0  0.656716  0.509091  0.366013  0.582904


,FAVOR,AGAINST,NONE,AVG
0,0.656716,0.509091,0.366013,0.582904


In [85]:
get_f1(test_taskA)

     FAVOR   AGAINST      NONE       AVG
0  0.72643  0.592664  0.537423  0.659547


,FAVOR,AGAINST,NONE,AVG
0,0.72643,0.592664,0.537423,0.659547


In [87]:
test_taskA['opinion_key']=test_taskA['Opinion Towards'].apply(lambda x: x[0])

In [89]:
test_taskA.to_csv('gpttaskA.csv')

In [88]:
opinion_target=test_taskA[test_taskA['opinion_key']=='1'].reset_index(drop=True)
not_opinion_target=test_taskA[test_taskA['opinion_key']=='2'].reset_index(drop=True)
f1_opinion=f1_score(opinion_target['predicted_stance'],opinion_target['Stance'],average=None).reshape(1,-1)
f1_opinion_df = pd.DataFrame(f1_opinion,columns=['Favor','Against','None'],index=['Opinion'])
f1_opinion_df['Average']=(f1_opinion_df['Favor']+f1_opinion_df['Against'])/2
f1_not_opinion=f1_score(not_opinion_target['predicted_stance'],not_opinion_target['Stance'],average=None).reshape(1,-1)
f1_not_opinion_df = pd.DataFrame(f1_not_opinion,columns=['Favor','Against','None'],index=['not_opinion'])
f1_not_opinion_df['Average']=(f1_not_opinion_df['Favor']+f1_not_opinion_df['Against'])/2
print(f1_opinion_df)
print(f1_not_opinion_df)

           Favor   Against  None   Average
Opinion  0.64268  0.752108   0.0  0.697394
               Favor  Against      None  Average
not_opinion  0.41629  0.45283  0.746939  0.43456


## TaskB

In [30]:
def classify_stance_zero_shot(tweet, target):
    # Define the prompt for stance detection
    prompt = f"Detect the stance for the tweet about '{target}', if the author of the given tweet supports '{target}', please label it as 'FAVOR'.\
    If the author of the given tweet is against '{target}', please label it as 'AGAINST'.\
    If the author of the given tweet has a neutral stance or no stance about {target}, please label it as 'NONE'.\
    \nNow detect the stance of the author of this tweet using excatly one word from 'FAVOR','AGAINST','NONE': \
    \ntweet: {tweet}. The author's stance towards {target} is:"

    # Make an API call to GPT-3.5 Turbo for stance detection
    response = openai.Completion.create(
        engine="text-davinci-003",  # Use text-davinci-003 for GPT-3.5 Turbo
        prompt=prompt,
        max_tokens=100,  # Adjust max_tokens based on your desired output length
        temperature=0.5,  # Adjust temperature for creativity vs. determinism
        stop=None  # Add custom stop words if needed
    )

    # Extract the generated stance label from the API response
    generated_stance = response['choices'][0]['text'].strip()

    return generated_stance

# Apply the classify_stance function to each row in the dataframe
#test['predicted_stance'] = test.apply(lambda row: classify_stance(row['tweet'], row['target']), axis=1)

# Convert stance labels to lowercase for consistency
#df['predicted_stance'] = df['predicted_stance'].str.lower()

In [31]:
test_taskB = test[test['Target']=='Donald Trump'].reset_index(drop=True)

In [32]:
test_taskB

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,@2014voteblue @ChrisJZullo blindly supporting ...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
1,@ThePimpernelX @Cameron_Gray @CalebHowe Total...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
2,@JeffYoung @ThePatriot143 I fully support full...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,pos
3,@ABC Stupid is as stupid does! Showedhis true ...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@HouseGOP we now have one political party. The...,Donald Trump,NONE,2. The tweet does NOT expresses opinion about ...,neg
...,...,...,...,...,...
702,@realDonaldTrump we all want you as the next p...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
703,@RSherman_25 Join Twitter Trump brigade #oneth...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
704,@JoeyBats19 Join Twitter Trump brigade #onetho...,Donald Trump,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
705,Trump's outlandish statements is political str...,Donald Trump,AGAINST,1. The tweet explicitly expresses opinion abo...,neg


In [33]:
test_taskB['opinion_key']=test_taskB['Opinion Towards'].apply(lambda x: x[0])

In [36]:
test_taskB['predicted_stance'] = test_taskB.apply(lambda row: classify_stance_zero_shot(row['Tweet'], row['Target']), axis=1)

In [39]:
test_taskB['predicted_stance']=test_taskB['predicted_stance'].str.replace('.','')

C:\Users\renzehao\AppData\Local\Temp\ipykernel_55904\1401311131.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_taskB['predicted_stance']=test_taskB['predicted_stance'].str.replace('.','')


In [43]:
get_f1(test_taskB)

      FAVOR   AGAINST      NONE       AVG
0  0.771605  0.479419  0.726736  0.625512


,FAVOR,AGAINST,NONE,AVG
0,0.771605,0.479419,0.726736,0.625512


In [44]:
test_taskB.to_csv('gpttaskB.csv')

In [45]:
opinion_target=test_taskB[test_taskB['opinion_key']=='1'].reset_index(drop=True)
not_opinion_target=test_taskB[test_taskB['opinion_key']=='2'].reset_index(drop=True)
f1_opinion=f1_score(opinion_target['predicted_stance'],opinion_target['Stance'],average=None).reshape(1,-1)
f1_opinion_df = pd.DataFrame(f1_opinion,columns=['Favor','Against','None'],index=['Opinion'])
f1_opinion_df['Average']=(f1_opinion_df['Favor']+f1_opinion_df['Against'])/2
f1_not_opinion=f1_score(not_opinion_target['predicted_stance'],not_opinion_target['Stance'],average=None).reshape(1,-1)
f1_not_opinion_df = pd.DataFrame(f1_not_opinion,columns=['Favor','Against','None'],index=['not_opinion'])
f1_not_opinion_df['Average']=(f1_not_opinion_df['Favor']+f1_not_opinion_df['Against'])/2
print(f1_opinion_df)
print(f1_not_opinion_df)

           Favor   Against      None   Average
Opinion  0.59144  0.821192  0.044944  0.706316
                Favor   Against      None   Average
not_opinion  0.298701  0.090909  0.817164  0.194805


In [116]:
test_taskB.groupby(['opinion_key','Stance']).count()

Tweet  Target  Opinion Towards  Sentiment  \
opinion_key Stance                                               
1           AGAINST    176     176              176        176   
            FAVOR      145     145              145        145   
            NONE         3       3                3          3   
2           AGAINST    122     122              122        122   
            FAVOR        3       3                3          3   
            NONE       231     231              231        231   
3           AGAINST      1       1                1          1   
            NONE        26      26               26         26   

                     predicted_stance  predicted_stance_1  
opinion_key Stance                                         
1           AGAINST               176                 176  
            FAVOR                 145                 145  
            NONE                    3                   3  
2           AGAINST               122                 122  
            FAVOR                   3                   3  
            NONE                  231                 231  
3           AGAINST                 1                   1  
            NONE                   26                  26